# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 13 2023 11:21AM,255008,10,SOTAH0000885,"Nextsource Biotechnology, LLC",Released
1,Jan 13 2023 11:06AM,255007,19,MSP217533,"Methapharm, Inc.",Released
2,Jan 13 2023 11:06AM,255007,19,MSP217544,"Methapharm, Inc.",Released
3,Jan 13 2023 11:06AM,255007,19,MSP217336,Methapharm-G,Released
4,Jan 13 2023 11:06AM,255007,19,MSP217370,Methapharm-G,Released
5,Jan 13 2023 10:58AM,255005,10,0086298109,ISDIN Corporation,Released
6,Jan 13 2023 10:58AM,255005,10,0086298112,ISDIN Corporation,Released
7,Jan 13 2023 10:58AM,255005,10,0086298113,ISDIN Corporation,Released
8,Jan 13 2023 10:58AM,255005,10,0086298114,ISDIN Corporation,Released
9,Jan 13 2023 10:58AM,255005,10,0086298115,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,255004,Executing,2
20,255004,Released,3
21,255005,Released,16
22,255007,Released,4
23,255008,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
255002,NaN,NaN,1.0
255004,NaN,2.0,3.0
255005,NaN,NaN,16.0
255007,NaN,NaN,4.0
255008,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254918,0.0,0.0,1.0
254922,0.0,1.0,0.0
254949,0.0,0.0,4.0
254961,0.0,1.0,0.0
254976,0.0,9.0,39.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254918,0,0,1
254922,0,1,0
254949,0,0,4
254961,0,1,0
254976,0,9,39


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254918,0,0,1
1,254922,0,1,0
2,254949,0,0,4
3,254961,0,1,0
4,254976,0,9,39


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254918,,,1
1,254922,,1,
2,254949,,,4
3,254961,,1,
4,254976,,9,39


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 13 2023 11:21AM,255008,10,"Nextsource Biotechnology, LLC"
1,Jan 13 2023 11:06AM,255007,19,"Methapharm, Inc."
3,Jan 13 2023 11:06AM,255007,19,Methapharm-G
5,Jan 13 2023 10:58AM,255005,10,ISDIN Corporation
21,Jan 13 2023 10:54AM,255004,10,ISDIN Corporation
26,Jan 13 2023 10:39AM,255002,19,"AdvaGen Pharma, Ltd"
27,Jan 13 2023 10:07AM,254999,10,Eywa Pharma Inc.
33,Jan 13 2023 10:01AM,254996,15,"Virtus Pharmaceuticals, LLC"
38,Jan 13 2023 10:01AM,254995,15,"Mizner Bioscience, LLC"
39,Jan 13 2023 9:45AM,254997,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 13 2023 11:21AM,255008,10,"Nextsource Biotechnology, LLC",,,1
1,Jan 13 2023 11:06AM,255007,19,"Methapharm, Inc.",,,4
2,Jan 13 2023 11:06AM,255007,19,Methapharm-G,,,4
3,Jan 13 2023 10:58AM,255005,10,ISDIN Corporation,,,16
4,Jan 13 2023 10:54AM,255004,10,ISDIN Corporation,,2,3
5,Jan 13 2023 10:39AM,255002,19,"AdvaGen Pharma, Ltd",,,1
6,Jan 13 2023 10:07AM,254999,10,Eywa Pharma Inc.,,,6
7,Jan 13 2023 10:01AM,254996,15,"Virtus Pharmaceuticals, LLC",,,5
8,Jan 13 2023 10:01AM,254995,15,"Mizner Bioscience, LLC",,,1
9,Jan 13 2023 9:45AM,254997,10,Bio-PRF,,4,17


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 13 2023 11:21AM,255008,10,"Nextsource Biotechnology, LLC",1,,
1,Jan 13 2023 11:06AM,255007,19,"Methapharm, Inc.",4,,
2,Jan 13 2023 11:06AM,255007,19,Methapharm-G,4,,
3,Jan 13 2023 10:58AM,255005,10,ISDIN Corporation,16,,
4,Jan 13 2023 10:54AM,255004,10,ISDIN Corporation,3,2,
5,Jan 13 2023 10:39AM,255002,19,"AdvaGen Pharma, Ltd",1,,
6,Jan 13 2023 10:07AM,254999,10,Eywa Pharma Inc.,6,,
7,Jan 13 2023 10:01AM,254996,15,"Virtus Pharmaceuticals, LLC",5,,
8,Jan 13 2023 10:01AM,254995,15,"Mizner Bioscience, LLC",1,,
9,Jan 13 2023 9:45AM,254997,10,Bio-PRF,17,4,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 13 2023 11:21AM,255008,10,"Nextsource Biotechnology, LLC",1,,
1,Jan 13 2023 11:06AM,255007,19,"Methapharm, Inc.",4,,
2,Jan 13 2023 11:06AM,255007,19,Methapharm-G,4,,
3,Jan 13 2023 10:58AM,255005,10,ISDIN Corporation,16,,
4,Jan 13 2023 10:54AM,255004,10,ISDIN Corporation,3,2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 13 2023 11:21AM,255008,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
1,Jan 13 2023 11:06AM,255007,19,"Methapharm, Inc.",4.0,NaN,NaN
2,Jan 13 2023 11:06AM,255007,19,Methapharm-G,4.0,NaN,NaN
3,Jan 13 2023 10:58AM,255005,10,ISDIN Corporation,16.0,NaN,NaN
4,Jan 13 2023 10:54AM,255004,10,ISDIN Corporation,3.0,2.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 13 2023 11:21AM,255008,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
1,Jan 13 2023 11:06AM,255007,19,"Methapharm, Inc.",4.0,0.0,0.0
2,Jan 13 2023 11:06AM,255007,19,Methapharm-G,4.0,0.0,0.0
3,Jan 13 2023 10:58AM,255005,10,ISDIN Corporation,16.0,0.0,0.0
4,Jan 13 2023 10:54AM,255004,10,ISDIN Corporation,3.0,2.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2039932,52.0,11.0,3.0
12,254961,0.0,1.0,0.0
15,1019977,21.0,62.0,0.0
19,1019992,48.0,9.0,0.0
21,509840,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2039932,52.0,11.0,3.0
1,12,254961,0.0,1.0,0.0
2,15,1019977,21.0,62.0,0.0
3,19,1019992,48.0,9.0,0.0
4,21,509840,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,52.0,11.0,3.0
1,12,0.0,1.0,0.0
2,15,21.0,62.0,0.0
3,19,48.0,9.0,0.0
4,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,52.0
1,12,Released,0.0
2,15,Released,21.0
3,19,Released,48.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,21
Status,,,,,
Completed,3.0,0.0,0.0,0.0,0.0
Executing,11.0,1.0,62.0,9.0,1.0
Released,52.0,0.0,21.0,48.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,21
0,Completed,3.0,0.0,0.0,0.0,0.0
1,Executing,11.0,1.0,62.0,9.0,1.0
2,Released,52.0,0.0,21.0,48.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,21
0,Completed,3.0,0.0,0.0,0.0,0.0
1,Executing,11.0,1.0,62.0,9.0,1.0
2,Released,52.0,0.0,21.0,48.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()